# Image Formation
In this notebook we are going to cover the theory behind the pinhole camera model, the perspective projection equation and the distortion parameters.

In [1]:
%matplotlib widget

from ipywidgets import interact, FloatSlider, Checkbox
import ipywidgets as widgets

import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib.patches import Ellipse, Polygon
import matplotlib.lines as lines
import matplotlib.patches as mpatches
from matplotlib.collections import PolyCollection

## Pinhole Camera Model
### Thin Lens Equation



In [2]:
class ThinLensSchema:
    """
    This class is used throughout this notebook for visualizing formulas for the thin lens
    """
    
    def __init__(self):
        self.max_A = 10; self.max_Z = 50; self.max_f = 5.0
        self.min_A = 5; self.min_Z = 10; self.min_f = 1.0
        
        self.neg_x_lim = self.min_Z*self.max_f/(self.min_Z-self.max_f)
    
        self.fig = plt.figure(figsize=(9.5,4))
        self.fig.tight_layout()
        self.fig.canvas.toolbar_visible = False
        self.fig.canvas.header_visible = False
        self.fig.canvas.footer_visible = False
        
        self.ax = self.fig.gca()
        self.ax.set_title("Thin Lens Equation")
        self.ax.set_xlim([-self.neg_x_lim, self.max_Z])
        self.ax.set_ylim([-self.max_A, self.max_A])
        self.ax.set_aspect('equal')
        # Coodinate system is a bit weird since z should point to the right: (z,x)
        self.ax.set_xlabel('z') 
        self.ax.set_ylabel('x')
        self.ax.grid(True)
        
        self.focal_length = self.max_f
        self.sensor_distance = self.focal_length
        
        self.ax.add_artist(Ellipse((0, 0), 1, 2*self.max_A, color='b', alpha=0.2))
        
        self.house, = self.ax.plot([], [], color='r')
        self.sensor, = self.ax.plot([], [], color='k', linewidth=3)
        self.focal_point, = self.ax.plot([], [], 'ko')
        
        self.characteristic_ray_center, = self.ax.plot([], [], color='g')
        self.characteristic_ray_focus, = self.ax.plot([], [], color='g')
        
        self.aperture_top, = self.ax.plot([], [], color='k', linewidth=3)
        self.aperture_bot, = self.ax.plot([], [], color='k', linewidth=3)
        self.ray_bundle = self.ax.add_collection(PolyCollection([], closed=True, alpha=0.2, color='y'));
        
        self.thin_lens_deriv_1, = self.ax.plot([], [], color='y', linestyle='dashed', linewidth=2)
        self.thin_lens_deriv_2, = self.ax.plot([], [], color='m', linestyle='dotted', linewidth=2)
        self.blur_circle_deriv, = self.ax.plot([], [], color='m', linestyle='dotted', linewidth=2)
    
    def focus(self, distance):
        Z = distance
        f = self.focal_length
        self.set_sensor_distance(Z*f/(Z-f)) # Thin lens equation
    
    def set_sensor_distance(self, e):
        self.sensor.set_data([-e, -e], [-5, 5])
        self.sensor_distance = e
        
    def set_focal_length(self, f):
        self.focal_point.set_data([-f],[0])
        self.focal_length = f
    
    def draw_house(self, Z, A):
        self.house.set_data([Z-A/3, Z+A/3, Z-A/3, Z+A/3, Z, Z-A/3, Z-A/3, Z+A/3, Z+A/3], 
                            [0, 0, A*2/3, A*2/3, A, A*2/3, 0, A*2/3, 0])

    def clear_house(self, index):
        self.houses[index].set_data([],[])
    
    def draw_characteristic_rays(self, Z, A):
        f = self.focal_length
        e = Z*f/(Z-f) # The true focused distance
        self.characteristic_ray_center.set_data([Z, 0, -self.neg_x_lim], [A, A, A-self.neg_x_lim*A/f])
        self.characteristic_ray_focus.set_data([Z, -self.neg_x_lim], [A, A-(Z+self.neg_x_lim)*A/Z])
    
    def clear_characteristic_rays(self):
        self.characteristic_rays.set_data([], []) 
        
    def draw_ray_bundle(self, Z, A, L):
        self.aperture_top.set_data([0, 0], [L/2, self.max_A])
        self.aperture_bot.set_data([0, 0], [-L/2, -self.max_A])
        f = self.focal_length
        e_true = Z*f/(Z-f) # true focused distance = horizontal poisiton of focus point
        a_true = -A/Z*e_true# vertical position of focus point
        e = self.sensor_distance
   
        self.ray_bundle.set_verts(np.array([[[Z, A], [0, L/2], [-e, L/2-e*(L/2-a_true)/e_true], 
                                             [-e, -L/2-e*(-L/2-a_true)/e_true],[0, -L/2], [Z, A]]]))
    
    def draw_blur_circle_derivation(self, Z, A, L):
        f = self.focal_length
        e_true = Z*f/(Z-f) # true focused distance = horizontal poisiton of focus point
        a_true = -A/Z*e_true# vertical position of focus point
        e = self.sensor_distance
        
        if e_true < e:   
            # Double triangle
            self.blur_circle_deriv.set_data([0, -e, -e, 0, 0], 
                                            [L/2, L/2-e*(L/2-a_true)/e_true, -e*A/Z, 0, L/2])
        else:
            # Single triangle
            self.blur_circle_deriv.set_data([-e, -e, -e_true, 0, 0, -e], 
                                            [-e*A/Z, L/2-e*(L/2-a_true)/e_true, a_true, 0, L/2, L/2-e*(L/2-a_true)/e_true])
        
    def draw_thin_lens_equation_derivation(self, f, Z, A):
        self.focus(Z)
        self.draw_characteristic_rays(Z, A)
        e = self.sensor_distance
        self.thin_lens_deriv_1.set_data([Z, Z, -e, -e, Z], [A, 0, 0, -e*A/Z, A])
        self.thin_lens_deriv_2.set_data([0, 0, -e, -e, 0], [A, 0, 0, -e*A/Z, A])
        
derivation_schema = ThinLensSchema()

def plotThinLensEquation(A, Z, f, e, L):
    derivation_schema.set_focal_length(f)
    #derivation_schema.focus(Z)
    derivation_schema.set_sensor_distance(e)
    derivation_schema.draw_house(Z, A)
    derivation_schema.draw_characteristic_rays(Z, A)
    derivation_schema.draw_ray_bundle(Z, A, L)
    derivation_schema.draw_blur_circle_derivation(Z, A, L)

A_widget = FloatSlider(5, min=derivation_schema.min_A, max=derivation_schema.max_A)
Z_widget = FloatSlider(10, min=derivation_schema.min_Z, max=derivation_schema.max_Z)
f_widget = FloatSlider(5, min=derivation_schema.min_f, max=derivation_schema.max_f)      
e_widget = FloatSlider(5, min=2*derivation_schema.min_f, max=2*derivation_schema.max_f)     
L_widget = FloatSlider(5, min=1, max=20)     

interact(plotThinLensEquation, A=A_widget, Z=Z_widget, f=f_widget, e=e_widget, L=L_widget);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=5.0, description='A', max=10.0, min=5.0), FloatSlider(value=10.0, desc…


### Aperture & Blur Circle

### From the Thin Lens Equation to the Pinhole Camera Model